In [30]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import requests
import pymongo

In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [ ]:
# define the url and establish the connection with splinter for later
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
def connection(url):
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    return soup

In [ ]:
# Create BeautifulSoup object
#url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
#response = requests.get(url)
#soup = bs(response.text,'html.parser')

In [ ]:
#paragraphs = soup.find_all('div', class_='list_text')
#paragraphs

In [ ]:
#scrap the html to find the latest news title and paragraph
url = 'https://mars.nasa.gov/news/'
soup = connection(url)
result = soup.find('div', class_='list_text')
news_title = result.a.text
news_p = result.find('div', class_='article_teaser_body').text
print('Latest article from the NASA')
print(f'Title : {news_title}')
print(f'Teaser : {news_p}')

In [ ]:
#scrap the html to find the current Featured Mars Image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
soup = connection(url)
src = soup.footer.a['data-fancybox-href']
featured_img_url = f'https://www.jpl.nasa.gov{src}'
print(featured_img_url)

In [59]:
#scrap the html to find Mars's facts and store it in a df, export to html
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
mars_df = tables[0]
mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [60]:
mars_df.rename({0:'Infos', 1:'Unit'}, axis=1,inplace=True)
mars_df['Infos'] = mars_df['Infos'].str.replace(':','')
f = open('data/mars_table.html','w', encoding='utf-8')
mars_df.to_html(buf = f, index=False)
f.close()
mars_table = mars_df.to_dict('records')
mars_table

[{'Infos': 'Equatorial Diameter', 'Unit': '6,792 km'},
 {'Infos': 'Polar Diameter', 'Unit': '6,752 km'},
 {'Infos': 'Mass', 'Unit': '6.39 × 10^23 kg (0.11 Earths)'},
 {'Infos': 'Moons', 'Unit': '2 (Phobos & Deimos)'},
 {'Infos': 'Orbit Distance', 'Unit': '227,943,824 km (1.38 AU)'},
 {'Infos': 'Orbit Period', 'Unit': '687 days (1.9 years)'},
 {'Infos': 'Surface Temperature', 'Unit': '-87 to -5 °C'},
 {'Infos': 'First Record', 'Unit': '2nd millennium BC'},
 {'Infos': 'Recorded By', 'Unit': 'Egyptian astronomers'}]

In [ ]:
#Scrap high resoltution images for each hemisphere
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
soup = connection(url)
items = soup.find_all('div', class_='item')
hemisphere_image_urls=[]
for item in items:
    href = item.a['href']
    url = f'https://astrogeology.usgs.gov/{href}'
    browser.visit(url)
    html = browser.html
    soup = bs(html,'html.parser')
    url = soup.find('div',class_='downloads').\
        find('a')['href']
    title = soup.find('div', class_='content').\
        find('h2',class_='title').text 
    dict={}
    dict['title']=title
    dict['img_url']= url
    hemisphere_image_urls.append(dict)
hemisphere_image_urls

In [ ]:
#Scrap high resoltution images for each hemisphere
#url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
#soup = connection(url)
#items = soup.find_all('div', class_='item')
#hemisphere_image_urls=[]
#for item in items:
#    href = item.a['href']
#    browser.click_link_by_href(href)
#    html = browser.html
#    soup = bs(html,'html.parser')
#    url = soup.find('div',class_='downloads').\
#        find('a')['href']
#    title = soup.find('div', class_='content').\
#        find('h2',class_='title').text 
#    print(url)
#    print(title)